In [21]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os
import pprint
from dotenv import load_dotenv
load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = Chroma(persist_directory="./jonhWick_db", embedding_function=embeddings, collection_name="doc_jonhWick")


## To create a compression retriever, we need a base retriever first. In this case I will use the naive retriever (the simplest)

In [23]:
naive_retriever = vectordb.as_retriever(search_kwargs={ "k" : 10})

In [33]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')

compressor = CohereRerank(top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=naive_retriever
)

## We are going to do a Naive RAG.

## Remember:

- R -> Retrieval
- A -> Augmented
- G -> Generation

# Retrieval

In [25]:
# We have already created the retriever object
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x00000170F2ADD570>, top_n=3, model='rerank-english-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000170F05C9060>, search_kwargs={'k': 10}))

# Augmented

In [26]:
from langchain_core.prompts import ChatPromptTemplate

TEMPLATE = """\
You are happy assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

# Generation

In [27]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

## Finally, we are going to create a Reranking Retrieval. For that, we are going to use LCEL (LangChain Expression Language)

In [28]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compression_retriever })
output_parser = StrOutputParser()


compressor_retrieval_chain = setup_and_retrieval | rag_prompt | chat_model | output_parser


compressor_retrieval_chain.invoke( "Did people generally like John Wick?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Did people generally like John Wick?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context>] Entering Chain run with input:
{
  "input": "Did people generally like John Wick?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context> > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Did people generally like John Wick?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context> > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Did people generally like John Wick?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context>] [1.33s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 6:prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 6:prompt:ChatP

'Yes, people generally liked John Wick.'

### Naive retriever VS Contextual Compression Retriever

In [38]:
naive_retriever.invoke("What are the most positive reviews?")

[Document(page_content=': 12\nReview: The only word that keeps coming back to mind when reviewing this movie is: exhilarating. The fun factor this amazing action film has is probably the highest of any movie this year. I could not stop myself from having a blast!', metadata={'Author': 'Giacomo_De_Bello', 'Movie_Title': 'John Wick 1', 'Rating': 8, 'Review_Date': '23 January 2015', 'Review_Title': ' 8/10\n', 'Review_Url': '/review/rw3168672/?ref_=tt_urv', 'row': 12, 'source': 'data/john_wick_1.csv'}),
 Document(page_content=": 8\nReview: It's hard to find anything bad to say about John Wick. The action is beautifully choreographed, the setup is surprisingly emotional for an action flick, and Keanu.... What more is there to say? If you love action or even just like it you will be in for the ride of your life.", metadata={'Author': 'MrHeraclius', 'Movie_Title': 'John Wick 1', 'Rating': 5, 'Review_Date': '23 February 2020', 'Review_Title': ' love this movie highly recommend\n', 'Review_Url'

In [39]:
compression_retriever.invoke("What are the most positive reviews?")

[Document(page_content=": 16\nReview: John Wick 3 is without a doubt the best action movie to have come out in a few years. And its so surprising and refreshing to see that movies like this still exist. Most action movies you see is filled with ridiculous amounts of shaky-cam, fast edits and way over-the-top fights. This is perfect. You can see all action clear as day, and by god the stuns in this film are extraordinary. Why the Academy Awards don't award talent like this is beyond me. There were times that I wondered if they had used CGI, because someone nuts must be willing to throw themselves from motorbikes and through glass. But whatever case it is, its a dam joy to watch.", metadata={'Author': 'Dannyboi94', 'Movie_Title': 'John Wick 3', 'Rating': 8, 'Review_Date': '16 May 2019', 'Review_Title': " Finally an action franchise than doesn't lose its touch!\n", 'Review_Url': '/review/rw4858493/?ref_=tt_urv', 'row': 16, 'source': 'data/john_wick_3.csv', 'relevance_score': 0.40727445}),